In [1]:
%matplotlib inline

## 處理數據
PyTorch 有兩個[處理數據的方式](https://pytorch.org/docs/stable/data.html)：“torch.utils.data.DataLoader”和“torch.utils.data.Dataset” `. ``Dataset`` 存儲樣本及其相應的標籤，``DataLoader`` 圍繞 ``Dataset`` 包裝一個可迭代對象。



In [2]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor, Lambda, Compose
import matplotlib.pyplot as plt

PyTorch 提供特定領域的庫，例如 [TorchText](https://pytorch.org/text/stable/index.html)、[TorchVision](https://pytorch.org/vision/stable/index.html)、 和 [TorchAudio](https://pytorch.org/audio/stable/index.html)，所有這些都包含數據集。在本教程中，我們將使用 TorchVision 數據集。

``torchvision.datasets`` 模塊包含許多真實世界視覺數據的 ``Dataset`` 對象，例如 CIFAR、COCO（[完整列表在這裡]（https://pytorch.org/docs/stable/torchvision/datasets. html))。在本教程中，我們將使用 **FashionMNIST** 數據集。每個 TorchVision ``Dataset`` 包括兩個參數：``transform`` 和 ``target_transform`` 分別用於修改樣本和標籤。



In [3]:
# Download training data from open datasets.
training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor(),
)

# Download test data from open datasets.
test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor(),
)

100.0%

Extracting data/FashionMNIST/raw/train-images-idx3-ubyte.gz to data/FashionMNIST/raw


111.0%

Extracting data/FashionMNIST/raw/train-labels-idx1-ubyte.gz to data/FashionMNIST/raw


100.0%

Extracting data/FashionMNIST/raw/t10k-images-idx3-ubyte.gz to data/FashionMNIST/raw


159.1%

Extracting data/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to data/FashionMNIST/raw
Processing...
Done!


我們將“Dataset”作為參數傳遞給“DataLoader”。這在我們的數據集上包裝了一個迭代，並支持自動批處理、採樣、混洗和多進程數據加載。這裡我們定義了 64 的批量大小，即數據加載器迭代中的每個元素將返回一批 64 個特徵和標籤。



In [4]:
batch_size = 64

# Create data loaders.
train_dataloader = DataLoader(training_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)

for X, y in test_dataloader:
    print("Shape of X [N, C, H, W]: ", X.shape)
    print("Shape of y: ", y.shape, y.dtype)
    break

Shape of X [N, C, H, W]:  torch.Size([64, 1, 28, 28])
Shape of y:  torch.Size([64]) torch.int64


## 創建模型
為了在 PyTorch 中定義神經網絡，我們創建了一個繼承自 [nn.Module](https://pytorch.org/docs/stable/generated/torch.nn.Module.html) 的類。我們在 __init__ 函數中定義網絡層，並在 forward 函數中指定數據如何通過網絡。為了加速神經網絡中的操作，我們將其移至 GPU（如果可用）。



In [5]:
# Get cpu or gpu device for training.
device = "cuda" if torch.cuda.is_available() else "cpu"
print("Using {} device".format(device))

# Define model
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
            nn.ReLU()
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork().to(device)
print(model)

Using cpu device
NeuralNetwork(
  (flatten): Flatten()
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
    (5): ReLU()
  )
)


## 優化模型參數
為了訓練模型，我們需要一個[損失函數](https://pytorch.org/docs/stable/nn.html#loss-functions>)和一個[優化器](https://pytorch.org/docs/stable/optim.html）。



In [6]:
loss_fn = nn.CrossEntropyLoss()
learning_rate = 1e-3
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

在單個訓練循環中，模型對訓練數據集進行預測（分批提供給它），並反向傳播預測誤差以調整模型的參數。 


In [7]:
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)
        
        # Compute prediction error
        pred = model(X)
        loss = loss_fn(pred, y)
        
        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

我们还可以根据测试数据集检查模型的性能，以确保它正在学习。

In [8]:
def test(dataloader, model):
    size = len(dataloader.dataset)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= size
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

訓練過程經過多次迭代（*epochs*）。在每個時期，模型學習參數以做出更好的預測。我們在每個時期打印模型的準確性和損失； 我們希望看到準確率隨著每個 epoch 的增加而減少。



In [9]:
epochs = 15
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model)
print("Done!")

Epoch 1
-------------------------------
loss: 2.305339  [    0/60000]
loss: 2.298571  [ 6400/60000]
loss: 2.283322  [12800/60000]
loss: 2.289912  [19200/60000]
loss: 2.263478  [25600/60000]
loss: 2.235991  [32000/60000]
loss: 2.261578  [38400/60000]
loss: 2.223481  [44800/60000]
loss: 2.227989  [51200/60000]
loss: 2.219072  [57600/60000]
Test Error: 
 Accuracy: 43.3%, Avg loss: 0.034741 

Epoch 2
-------------------------------
loss: 2.211195  [    0/60000]
loss: 2.223341  [ 6400/60000]
loss: 2.175561  [12800/60000]
loss: 2.215628  [19200/60000]
loss: 2.154950  [25600/60000]
loss: 2.090364  [32000/60000]
loss: 2.164232  [38400/60000]
loss: 2.072783  [44800/60000]
loss: 2.097270  [51200/60000]
loss: 2.093919  [57600/60000]
Test Error: 
 Accuracy: 46.4%, Avg loss: 0.032619 

Epoch 3
-------------------------------
loss: 2.074818  [    0/60000]
loss: 2.100671  [ 6400/60000]
loss: 2.004231  [12800/60000]
loss: 2.085694  [19200/60000]
loss: 1.969844  [25600/60000]
loss: 1.860817  [32000/600

最初的準確性不會很好（沒關係！）。嘗試運行循環以獲得更多的“epochs”或將“learning_rate”調整為更大的數字。也可能是我們選擇的模型配置可能不是此類問題的最佳配置（事實並非如此）。後面的課程將深入研究適用於視力問題的模型形狀。

保存模型
-------------
保存模型的常用方法是序列化內部狀態字典（包含模型參數）。



In [10]:
torch.save(model.state_dict(), "data/model.pth")
print("Saved PyTorch Model State to model.pth")

Saved PyTorch Model State to model.pth


加載模型
-----------------------------

加載模型的過程包括重新創建模型結構和加載
將狀態字典放入其中。



In [11]:
model = NeuralNetwork()
model.load_state_dict(torch.load("data/model.pth"))

<All keys matched successfully>

該模型現在可用於進行預測。



In [12]:
classes = [
    "T-shirt/top",
    "Trouser",
    "Pullover",
    "Dress",
    "Coat",
    "Sandal",
    "Shirt",
    "Sneaker",
    "Bag",
    "Ankle boot",
]

model.eval()
x, y = test_data[0][0], test_data[0][1]
with torch.no_grad():
    pred = model(x)
    predicted, actual = classes[pred[0].argmax(0)], classes[y]
    print(f'Predicted: "{predicted}", Actual: "{actual}"')

Predicted: "Ankle boot", Actual: "Ankle boot"
